In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import psycopg2

# settings
datapath = '/Users/mirkoknoche/MIMIC/'
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,}'.format
%matplotlib inline

# database
conn_string = open('./intern/DB_connection.txt', 'r', encoding='UTF-8').read()

# Selecting the patient

We filter the patients in two steps.   
First we select from all admissions those who are **diagnosed with ARDS**:  


# Selecting features

selecting label

## Find extubations

In [ ]:
conn = eval('psycopg2.connect({})'.format(conn_string.strip()))

In [ ]:
sql = """select i2.hadm_id, v.* 
        from ventsettings v
        inner join icustays i2 on i2.icustay_id = v.icustay_id 
        where (v.mechvent + v.oxygentherapy + v.extubated + v.selfextubated) > 0
        order by icustay_id, charttime """
df = pd.read_sql(sql, conn)

In [ ]:
df.shape

In [ ]:
df.head(3)

In [ ]:
df.info()

Checking the exturbations

In [ ]:
df_count_ex = df.groupby('icustay_id').extubated.sum()

In [ ]:
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(1, 1, 1)
ax.hist(df_count_ex)
plt.ylabel('Amount of ICU-stays')
plt.xlabel('Extubations')
plt.title('Histrogram of the extubations')
plt.savefig('./images/hist_extubations_per_icustay.png')
plt.show();

Many ICU stays have no explicit documented extubation. We need to extract the id's and find another way.

In [ ]:
err_ext = list(df_count_ex[df_count_ex == 0].index)
len(err_ext)

In [ ]:
sql = """select * from transfers"""
df_icuunit = pd.read_sql(sql, conn)

# Filtering the icustay_id's with no extubations
df_icuunit = df_icuunit[df_icuunit.icustay_id.isin(err_ext)]

# use only the interesting columns
df_icuunit = df_icuunit[['icustay_id', 'curr_careunit']]

# drop duplicates
df_icuunit.drop_duplicates(inplace=True)

In [ ]:
df_plot = df_icuunit.groupby('curr_careunit').count()

fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(1, 1, 1)
ax.bar(list(df_plot.index), height=list(df_plot.values.flatten()))
plt.ylabel('Missing extubations')
plt.xlabel('ICU units')
plt.title('Not documented extubations')
plt.savefig('./images/missing_extubations_per_unit.png')
plt.show();

Have a look on one patient.

In [ ]:
df_qry = df[df.icustay_id == 200063]
df_qry.head()

In [ ]:
# getting the changes
df_qry['extube'] = df_qry['mechvent'].diff()
df_qry['oxy'] = df_qry['oxygentherapy'].diff()

In [ ]:
df_qry.extube.fillna(0, inplace=True)
df_qry.oxy.fillna(0, inplace=True)

In [ ]:
df_qry.loc[160:180]

In [ ]:
df_qry[(df_qry['extube'] != 0) | (df_qry['oxy'] != 0)]

In [ ]:
conn.close()